# TDT4225 Excercise 2 - Part 2

In [1]:
from DbConnector import DbConnector
import SqlQueriespt2 as SQ2

## Question 1: How many users, activities, and trackpoints are there in the dataset


In [2]:
db = DbConnector()

count_users_query = "SELECT COUNT(*) FROM User"
count_activities_query = "SELECT COUNT(*) FROM Activity"
count_trackpoints_query = "SELECT COUNT(*) FROM TrackPoint"

SQ2.query_to_db(db, count_users_query)
SQ2.query_to_db(db, count_activities_query)
SQ2.query_to_db(db, count_trackpoints_query)
db.close_connection()

2023-10-04 13:59:41,138 SQL PART2  INFO Executing: SELECT COUNT(*) FROM User
2023-10-04 13:59:41,163 SQL PART2  INFO Executing: SELECT COUNT(*) FROM Activity
2023-10-04 13:59:41,190 SQL PART2  INFO Executing: SELECT COUNT(*) FROM TrackPoint


Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

Rows produced by statement 'SELECT COUNT(*) FROM User':
[(182,)]
Rows produced by statement 'SELECT COUNT(*) FROM Activity':
[(16048,)]
Rows produced by statement 'SELECT COUNT(*) FROM TrackPoint':
[(9681756,)]

-----------------------------------------------
Connection to 8.0.34-0ubuntu0.22.04.1 is closed


>NOTE! We do not count the original dataset here, we only count the part of the dataset that fits the requirements for the excercise. 

## Question 2: Find the average, maximum, and minimum number of trackpoints per user

In [ ]:

def statistics_trackpoints_per_user(function:str = "AVG"):
    string = f"""
        WITH 
            query AS (
                        SELECT COUNT(trackpoint_id) AS trackpoint_count, activity_id FROM TrackPoint
                        GROUP BY activity_id
                    ),
            activities AS (
                SELECT user_id, activity_id FROM Activity
                ),
            joined_user AS (SELECT activities.user_id, query.trackpoint_count FROM query JOIN activities ON query.activity_id = activities.activity_id)

            SELECT user_id, {function}(trackpoint_count) FROM joined_user
            GROUP BY user_id
        ;
        """
    return string

db = DbConnector()
SQ2.query_to_db(db, statistics_trackpoints_per_user("AVG"))
SQ2.query_to_db(db, statistics_trackpoints_per_user("MIN"))
SQ2.query_to_db(db, statistics_trackpoints_per_user("MAX"))
db.close_connection



>NOTE:
> 
>First value in returned tuple is `user_id` second value is the result of the passed function. 
>
> *Still only using DB values*

## Question 3: Find the top 15 users with the highest number of activities

In [10]:
query_top_15_active_users = """
SELECT user_id, COUNT(activity_id) AS result FROM Activity
GROUP BY user_id
ORDER BY result DESC
LIMIT 15
;
"""

db = DbConnector()
SQ2.query_to_db(db, query_top_15_active_users)
db.close_connection

2023-10-04 15:41:28,606 SQL PART2  INFO Executing: 
SELECT user_id, COUNT(activity_id) AS result FROM Activity
GROUP BY user_id
ORDER BY result DESC
LIMIT 15
;



Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('main_db',)
-----------------------------------------------

Rows produced by statement 'SELECT user_id, COUNT(activity_id) AS result FROM Activity
GROUP BY user_id
ORDER BY result DESC
LIMIT 15':
[('128', 2102), ('153', 1793), ('025', 715), ('163', 704), ('062', 691), ('144', 563), ('041', 399), ('085', 364), ('004', 346), ('140', 345), ('167', 320), ('068', 280), ('017', 265), ('003', 261), ('014', 236)]


<bound method DbConnector.close_connection of <DbConnector.DbConnector object at 0x10abd0a60>>